In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created and saved")

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [5]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...


True

In [6]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

121 documents
43 classes ['HR_related_problem', 'Location', 'Weather', 'about', 'appointment status', 'cabin', 'check_leave', 'commission', 'competitors_in_market', 'configuration', 'connect_people', 'cost_lowering', 'customer_satisfaction', 'domain', 'email_id', 'factors_impacting_sale', 'forgot_password', 'gadgets', 'goodbye', 'greeting', 'highest_grossing', 'hours', 'invalid', 'key_customers', 'leave', 'maintainence', 'manufacturing_problems', 'missing_id', 'name', 'noans', 'options', 'order_components', 'order_tracking', 'predict_delay', 'predict_performance', 'project_handling_queries', 'search_department', 'search_person_by_id', 'solve_problems', 'supplier_info', 'thanks', 'turnover', 'version_update']
241 unique lemmatized words ["'s", ',', '.', '23a12', '23a31', '32712', '345a23', '431b67', '561a24', '562b78', '@', 'a', 'abc', 'abx', 'accident', 'ai', 'am', 'an', 'analysis', 'and', 'anyone', 'appointment', 'appoitment', 'are', 'at', 'awesome', 'based', 'be', 'been', 'benefit', 

In [7]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [8]:
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\Hp\AppData\Local\Temp/ipykernel_17460/3824779042.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created and saved")

Epoch 1/200
25/25 [==============================] - 8s 15ms/step - loss: 3.7394 - accuracy: 0.0162
Epoch 2/200
25/25 [==============================] - 0s 6ms/step - loss: 3.6903 - accuracy: 0.0952
Epoch 3/200
25/25 [==============================] - 0s 6ms/step - loss: 3.5843 - accuracy: 0.0840
Epoch 4/200
25/25 [==============================] - 0s 3ms/step - loss: 3.5349 - accuracy: 0.1104
Epoch 5/200
25/25 [==============================] - 0s 3ms/step - loss: 3.3668 - accuracy: 0.1347
Epoch 6/200
25/25 [==============================] - 0s 3ms/step - loss: 3.3833 - accuracy: 0.1169
Epoch 7/200
25/25 [==============================] - 0s 3ms/step - loss: 2.9806 - accuracy: 0.1640
Epoch 8/200
25/25 [==============================] - 0s 3ms/step - loss: 2.9265 - accuracy: 0.1989
Epoch 9/200
25/25 [==============================] - 0s 3ms/step - loss: 2.8588 - accuracy: 0.2714
Epoch 10/200
25/25 [==============================] - 0s 3ms/step - loss: 2.6212 - accuracy: 0.4109
Epoch 11

25/25 [==============================] - 0s 3ms/step - loss: 0.3426 - accuracy: 0.8926
Epoch 84/200
25/25 [==============================] - 0s 3ms/step - loss: 0.3965 - accuracy: 0.8894
Epoch 85/200
25/25 [==============================] - 0s 3ms/step - loss: 0.3524 - accuracy: 0.8841
Epoch 86/200
25/25 [==============================] - 0s 3ms/step - loss: 0.3480 - accuracy: 0.9048
Epoch 87/200
25/25 [==============================] - 0s 3ms/step - loss: 0.2542 - accuracy: 0.9501
Epoch 88/200
25/25 [==============================] - 0s 3ms/step - loss: 0.2391 - accuracy: 0.9363
Epoch 89/200
25/25 [==============================] - 0s 3ms/step - loss: 0.3048 - accuracy: 0.8571
Epoch 90/200
25/25 [==============================] - 0s 3ms/step - loss: 0.1868 - accuracy: 0.9475
Epoch 91/200
25/25 [==============================] - 0s 3ms/step - loss: 0.4110 - accuracy: 0.8847
Epoch 92/200
25/25 [==============================] - 0s 3ms/step - loss: 0.3443 - accuracy: 0.8839
Epoch 93/200


25/25 [==============================] - 0s 3ms/step - loss: 0.1746 - accuracy: 0.9299
Epoch 165/200
25/25 [==============================] - 0s 3ms/step - loss: 0.1918 - accuracy: 0.9385
Epoch 166/200
25/25 [==============================] - 0s 3ms/step - loss: 0.3809 - accuracy: 0.8480
Epoch 167/200
25/25 [==============================] - 0s 3ms/step - loss: 0.1869 - accuracy: 0.9514
Epoch 168/200
25/25 [==============================] - 0s 7ms/step - loss: 0.1631 - accuracy: 0.9299
Epoch 169/200
25/25 [==============================] - 0s 6ms/step - loss: 0.2787 - accuracy: 0.8883
Epoch 170/200
25/25 [==============================] - 0s 6ms/step - loss: 0.1268 - accuracy: 0.9499
Epoch 171/200
25/25 [==============================] - 0s 6ms/step - loss: 0.1656 - accuracy: 0.9527
Epoch 172/200
25/25 [==============================] - 0s 6ms/step - loss: 0.1722 - accuracy: 0.9201
Epoch 173/200
25/25 [==============================] - 0s 6ms/step - loss: 0.1908 - accuracy: 0.9159
Epoc